<a href="https://colab.research.google.com/github/singhjasdeep13/csci166/blob/main/CSCI_166_FrozenLake_DirectEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frozen Lake w/ Value Iteration & Direct Evaluation

## Frozen Lake Domain Description

Frozen Lake is a simple grid-world environment where an agent navigates a frozen lake to reach a goal while avoiding falling into holes. The environment is represented as a grid, with each cell being one of the following:

* **S**: Starting position of the agent
* **F**: Frozen surface, safe to walk on
* **H**: Hole, falling into one ends the episode with a reward of 0
* **G**: Goal, reaching it ends the episode with a reward of 1

The agent can take four actions:

* **0: Left**
* **1: Down**
* **2: Right**
* **3: Up**

However, due to the slippery nature of the ice, the agent might not always move in the intended direction. There's a chance it moves perpendicular to the intended direction.




In [ ]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG



/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


The transition model for the Frozen Lake world describes how the agent's actions affect its movement and the resulting state transitions. Here's a breakdown of the key components:

**Actions:**

* The agent can choose from four actions:
    * 0: Left
    * 1: Down
    * 2: Right
    * 3: Up

**State Transitions:**

* **Intended Movement:** Ideally, the agent moves one cell in the chosen direction.
* **Slippery Ice:** Due to the slippery nature of the ice, there's a probability that the agent will move in a perpendicular direction instead of the intended one. The exact probabilities depend on the specific Frozen Lake environment configuration, but typically:
    * **Successful Move:** The agent moves in the intended direction with a high probability.
    * **Perpendicular Move:** The agent moves 90 degrees to the left or right of the intended direction with a lower probability.
* **Boundaries:** If the intended movement would take the agent outside the grid boundaries, it remains in its current position.
* **Holes:** If the agent lands on a hole ("H"), the episode ends, and it receives a reward of 0.
* **Goal:** If the agent reaches the goal ("G"), the episode ends, and it receives a reward of 1.




In [ ]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()
print ("State 14 Going Right: (s, a, r, Done)", env.P[14][2])

  (Up)
SFFF
FHFH
FFFH
HFFG

State 14 Going Right: (s, a, r, Done) [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]


# Value Iteration

In [ ]:
import gym
import numpy as np

def value_iteration(env, gamma=0.9, num_iterations=1000, theta=0.0001):
    """
    Implements the Value Iteration algorithm.

    Args:
        env: The OpenAI Gym environment.
        gamma: Discount factor.
        num_iterations: Number of iterations to run.

    Returns:
        The optimal value function and policy.
    """
    # Initialize value function and policy
    V = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n, dtype=int)

    for _ in range(num_iterations):
        delta = 0
        V_prev = V.copy()
        for s in range(env.observation_space.n):
            v = V[s]
            action_values = []  # Store Q values for all actions in this state

            for a in range(env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in env.P[s][a]:
                    q_value += prob * (reward + gamma * V_prev[next_state])
                action_values.append(q_value)

            # Update V[s] with the max Q value
            V[s] = max(action_values)
            # Update policy[s] with the action that maximizes Q value
            policy[s] = np.argmax(action_values)

            delta = max(delta, abs(v - V[s]))
        if delta < theta and 0:
            break
        V_prev = V.copy()

    return V, policy

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Apply student's Value Iteration
optimal_V, optimal_policy = value_iteration(env)
print (f"optimal policy= \n{optimal_policy.reshape((4,4))}\n optimal_V=\n{np.round(optimal_V.reshape((4,4)), 2)}")

# Evaluate student's solution (Optional)
def evaluate_policy(env, policy, num_episodes=100):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            state, reward, done, _ =  env.step(action)
            total_reward += reward
    return total_reward / num_episodes

average_reward = evaluate_policy(env, optimal_policy)
print("Average Reward:", average_reward)

optimal policy= 
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
 optimal_V=
[[0.07 0.06 0.07 0.06]
 [0.09 0.   0.11 0.  ]
 [0.15 0.25 0.3  0.  ]
 [0.   0.38 0.64 0.  ]]
Average Reward: 0.69


# Policy Extraction & Policy Evaluation

In [ ]:
def policy_extraction(env, V, gamma=0.9):
  policy = np.zeros(env.observation_space.n, dtype=int)
  for s in range(env.observation_space.n):
      v = V[s]
      action_values = []  # Store Q values for all actions in this state

      for a in range(env.action_space.n):
          q_value = 0
          for prob, next_state, reward, done in env.P[s][a]:
              q_value += prob * (reward + gamma * V[next_state])
          action_values.append(q_value)

      # Update policy[s] with the action that maximizes Q value
      policy[s] = np.argmax(action_values)
  return policy

def policy_evaluation(env, policy, gamma=0.9, num_iterations=1000, theta=0.0001):
    """
    Implements the Value Iteration algorithm.

    Args:
        env: The OpenAI Gym environment.
        gamma: Discount factor.
        num_iterations: Number of iterations to run.

    Returns:
        The optimal value function and policy.
    """
    # Initialize value function and policy
    V = np.zeros(env.observation_space.n)
    for _ in range(num_iterations):
        delta = 0
        V_prev = V.copy()
        for s in range(env.observation_space.n):
            v = V[s]
            a = policy[s]
            q_value = 0
            for prob, next_state, reward, done in env.P[s][a]:
                q_value += prob * (reward + gamma * V_prev[next_state])
            # Update V[s] with the max Q value
            V[s] = q_value

            delta = max(delta, abs(v - V[s]))
        if delta < theta and 0:
            break
        V_prev = V.copy()
    return V

In [ ]:
V_policy_evalution = policy_evaluation(env, optimal_policy)
print (f"V_policy_evalution=\n{V_policy_evalution}")
V_policy = policy_extraction(env, V_policy_evalution)
print (f"optimal policy= \n{optimal_policy.reshape((4,4))}\n V_policy=\n{V_policy.reshape((4,4))}")

V_policy_evalution=
[0.0688909  0.06141457 0.07440976 0.05580732 0.09185454 0.
 0.11220821 0.         0.14543635 0.24749695 0.29961759 0.
 0.         0.3799359  0.63902015 0.        ]
optimal policy= 
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
 V_policy=
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


# Reinforcement Learning

In [ ]:
env.reset(seed=42)
init_policy = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
print (f"optimal policy= \n{init_policy.reshape((4,4))}")

optimal policy= 
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


In [ ]:
def GenerateEpisodes(env, policy, num_episodes=5):
    total_reward = 0
    episodes = []
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        episode = []
        while not done:
            action = policy[state]
            newstate, reward, done, _ = env.step(action)
            episode.append((int(state), action, newstate, reward))
            state = newstate
        episodes.append(episode)
    return episodes

training_episodes = GenerateEpisodes(env, init_policy)
for i, e in enumerate(training_episodes):
  print (f"training_episode=({i=}):\n{e}")

training_episodes2 = GenerateEpisodes(env, init_policy, 1000)

training_episode=(i=0):
[(0, 0, 4, 0.0), (4, 0, 8, 0.0), (8, 3, 9, 0.0), (9, 1, 10, 0.0), (10, 0, 14, 0.0), (14, 1, 15, 1.0)]
training_episode=(i=1):
[(0, 0, 0, 0.0), (0, 0, 0, 0.0), (0, 0, 4, 0.0), (4, 0, 4, 0.0), (4, 0, 8, 0.0), (8, 3, 4, 0.0), (4, 0, 0, 0.0), (0, 0, 0, 0.0), (0, 0, 0, 0.0), (0, 0, 4, 0.0), (4, 0, 4, 0.0), (4, 0, 8, 0.0), (8, 3, 4, 0.0), (4, 0, 8, 0.0), (8, 3, 8, 0.0), (8, 3, 8, 0.0), (8, 3, 9, 0.0), (9, 1, 13, 0.0), (13, 2, 13, 0.0), (13, 2, 13, 0.0), (13, 2, 9, 0.0), (9, 1, 10, 0.0), (10, 0, 14, 0.0), (14, 1, 13, 0.0), (13, 2, 14, 0.0), (14, 1, 14, 0.0), (14, 1, 13, 0.0), (13, 2, 13, 0.0), (13, 2, 14, 0.0), (14, 1, 13, 0.0), (13, 2, 9, 0.0), (9, 1, 13, 0.0), (13, 2, 9, 0.0), (9, 1, 10, 0.0), (10, 0, 6, 0.0), (6, 0, 10, 0.0), (10, 0, 14, 0.0), (14, 1, 14, 0.0), (14, 1, 13, 0.0), (13, 2, 9, 0.0), (9, 1, 8, 0.0), (8, 3, 9, 0.0), (9, 1, 8, 0.0), (8, 3, 8, 0.0), (8, 3, 4, 0.0), (4, 0, 8, 0.0), (8, 3, 8, 0.0), (8, 3, 4, 0.0), (4, 0, 4, 0.0), (4, 0, 0, 0.0), (0, 0, 0, 0.0

# Direct Evaluation

## Evaluate Single Episode

In [ ]:
def EvaluateEpisode(env, e, V_DE, V_Counts, gamma=0.9):
    future_reward = 0
    for t in reversed(e):  # Iterate in reverse order
        future_reward = t[3] + gamma * future_reward
        V_DE[t[0]] = future_reward+V_DE[t[0]]
        V_Counts[t[0]] = V_Counts[t[0]]+1
    return V_DE, V_Counts

## Evaluate Episode 1



In [ ]:
V_DE = np.zeros((env.observation_space.n))
V_Counts = np.zeros((env.observation_space.n))
V_DE, V_Count = EvaluateEpisode(env, training_episodes[0], V_DE, V_Counts, 0.9)
V = np.where(V_Counts != 0, V_DE / V_Counts, 0)
print (f"V_DE=\n{V_DE.reshape((4,4))}")
print (f"V_Counts=\n{V_Counts.reshape((4,4))}")
print (f"V=\n{V.reshape((4,4))}")

V_DE=
[[0.59049 0.      0.      0.     ]
 [0.6561  0.      0.      0.     ]
 [0.729   0.81    0.9     0.     ]
 [0.      0.      1.      0.     ]]
V_Counts=
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 1. 0.]
 [0. 0. 1. 0.]]
V=
[[0.59049 0.      0.      0.     ]
 [0.6561  0.      0.      0.     ]
 [0.729   0.81    0.9     0.     ]
 [0.      0.      1.      0.     ]]


<ipython-input-9-000a7187090b>:4: RuntimeWarning: invalid value encountered in divide
  V = np.where(V_Counts != 0, V_DE / V_Counts, 0)


## Evaluate All Episodes

In [ ]:
V_DE = np.zeros((env.observation_space.n))
V_Counts = np.zeros((env.observation_space.n))
for e in training_episodes2:
    V_DE, V_Count = EvaluateEpisode(env, e, V_DE, V_Counts, 0.9)
V = np.where(V_Counts != 0, V_DE / V_Counts, 0)
print (f"V_DE=\n{V_DE.reshape((4,4))}")
print (f"V_Counts=\n{V_Counts.reshape((4,4))}")
print (f"V_DirectEvaluation=\n{np.round(V.reshape((4,4)),2)}")
print (f"optimal policy= \n{optimal_policy.reshape((4,4))}\n optimal_V=\n{np.round(optimal_V.reshape((4,4)), 2)}")

V_DE=
[[ 879.44773993   12.02033939   32.45370905    0.        ]
 [ 910.67996731    0.           63.889207      0.        ]
 [ 943.61931212  887.58499323  418.52594351    0.        ]
 [   0.         1065.13267976 1369.46029196    0.        ]]
V_Counts=
[[12819.   227.   503.     0.]
 [ 9779.     0.   646.     0.]
 [ 6632.  3600.  1432.     0.]
 [    0.  2876.  2159.     0.]]
V_DirectEvaluation=
[[0.07 0.05 0.06 0.  ]
 [0.09 0.   0.1  0.  ]
 [0.14 0.25 0.29 0.  ]
 [0.   0.37 0.63 0.  ]]
optimal policy= 
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
 optimal_V=
[[0.07 0.06 0.07 0.06]
 [0.09 0.   0.11 0.  ]
 [0.15 0.25 0.3  0.  ]
 [0.   0.38 0.64 0.  ]]


<ipython-input-10-440a84a75783>:5: RuntimeWarning: invalid value encountered in divide
  V = np.where(V_Counts != 0, V_DE / V_Counts, 0)


# Q-Learning

In [ ]:
import random

def q_learning(env, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    """
    I used ChatGPT to help optimize these functions using the structure of the
    value iteration function. The optimal policy and optimal Q values vary
    based on the randomized choices meant to explore the entire environment
    using an epsilon of 0.1

    Implements the Q-Learning algorithm with Epsilon-Greedy action selection.

    Args:
        env: The OpenAI Gym environment.
        num_episodes: Number of episodes to run.
        alpha: Learning rate.
        gamma: Discount factor.
        epsilon: Exploration rate.

    Returns:
        Q: The learned Q-value function.
        policy: The optimal policy derived from Q.
    """
    Q = np.zeros((env.observation_space.n, env.action_space.n))

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            # Epsilon-Greedy randomized action selection
            if random.randint(0, 1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            # Take the action
            next_state, reward, done, _ = env.step(action)

            # Update Q-value using Bellman equation
            best_next_action = np.argmax(Q[next_state])  # Max Q value for the next state
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])

            # Move to the next state
            state = next_state

    policy = np.argmax(Q, axis=1)
    return Q, policy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Q-Learning
Q, optimal_policy = q_learning(env)
print(f"Optimal Policy: \n{optimal_policy.reshape((4, 4))}")
print(f"Optimal Q-Values:\n{Q}")

# Optimal policy
average_reward = evaluate_policy(env, optimal_policy)
print("Average Reward:", average_reward)

Optimal Policy: 
[[0 3 1 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
Optimal Q-Values:
[[0.05533788 0.04856475 0.0497748  0.0464122 ]
 [0.03718335 0.03493501 0.03559337 0.04825605]
 [0.05455405 0.06231444 0.04745184 0.04277469]
 [0.01282524 0.02223893 0.01260682 0.04094245]
 [0.08280808 0.06060981 0.06219694 0.04723442]
 [0.         0.         0.         0.        ]
 [0.09516035 0.03511808 0.07915709 0.01340697]
 [0.         0.         0.         0.        ]
 [0.03721352 0.08773549 0.07671951 0.12437365]
 [0.122975   0.18958664 0.15301933 0.07323143]
 [0.32737138 0.20945979 0.1414324  0.03572877]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.12654909 0.13457577 0.3986834  0.34558009]
 [0.28842412 0.63923528 0.40951795 0.47002767]
 [0.         0.         0.         0.        ]]
Average Reward: 0.67


# Comparing the Q-Learning values to the Value Iteration values

```
Value Iteration values:

optimal policy=
[[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]
 optimal_V=
[[0.07 0.06 0.07 0.06]
 [0.09 0.   0.11 0.  ]
 [0.15 0.25 0.3  0.  ]
 [0.   0.38 0.64 0.  ]]
Average Reward: 0.69
```


```
Q-Learning values:

Optimal Policy:
[[0 3 0 1]
 [0 0 1 0]
 [3 1 0 0]
 [0 2 1 0]]
Optimal Q-Values:
[[0.07633568 0.06776639 0.07294079 0.05973072]
 [0.03932479 0.0360577  0.04771194 0.05959271]
 [0.06961512 0.06619583 0.06782806 0.053756  ]
 [0.01055646 0.04466839 0.01655398 0.0241352 ]
 [0.11523872 0.06910472 0.05657164 0.06350721]
 [0.         0.         0.         0.        ]
 [0.08215437 0.12019106 0.11604543 0.02791263]
 [0.         0.         0.         0.        ]
 [0.0735724  0.07015024 0.10810174 0.15213926]
 [0.11471993 0.22846509 0.20324377 0.13547533]
 [0.26964691 0.24076213 0.15962786 0.08372177]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.17035553 0.20893492 0.28754611 0.26027103]
 [0.31257376 0.55692456 0.3983626  0.37721044]
 [0.         0.         0.         0.        ]]
Average Reward: 0.64
```



Comparing Value Iteration and Q-Learning values we see a few differences. Q-Learning policies, values, and rewards all vary based on the random movements meant to explore the entire environment. Many times the rewards are less and the policy can be far from optimal. Sometimes it reaches close to the optimal values and policy provided through value iteration as seen above. Very rarely is the reward higher than value iteration even if the optimal policies match.

# Q-Learning with decaying epsilon

In [ ]:
import random

def q_learning2(env, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon_start=1.0, epsilon_end=0.1, decay_rate=0.5):
    """
    I am still struggling to understand this concept and used ChatGPT to help
    optimize this code. The only differences between this function and the
    original above is the starting and ending values of epsilon that is
    affected by decay rate, which in turn lowers the exploration rate of the
    agent after each episode. I have tested with decay_rate values ranging
    from 0.95 to 0.5 and the results vary each execution.

    Implements the Q-Learning algorithm with Epsilon-Greedy action selection.

    Args:
        env: The OpenAI Gym environment.
        num_episodes: Number of episodes to run.
        alpha: Learning rate.
        gamma: Discount factor.
        epsilon_start: Starting exploration rate.
        epsilon_end: Minimum exploration rate.
        decay_rate: Epsilon decay rate.

    Returns:
        Q: The learned Q-value function.
        policy: The optimal policy derived from Q.
    """
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    epsilon = epsilon_start # Set starting epsilon value

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            # Epsilon-Greedy randomized action selection
            if random.randint(0, 1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            # Take the action
            next_state, reward, done, _ = env.step(action)

            # Update Q-value using Bellman equation
            best_next_action = np.argmax(Q[next_state])  # Max Q value for the next state
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])

            # Move to the next state
            state = next_state

        # Epsilon will decay each episode
        epsilon = max(epsilon_end, epsilon * decay_rate)

    policy = np.argmax(Q, axis=1)
    return Q, policy

In [ ]:
# Q-Learning with decay
Q, optimal_policy = q_learning2(env)
print(f"Optimal Policy: \n{optimal_policy.reshape((4, 4))}")
print(f"Optimal Q-Values:\n{Q}")

# Optimal policy
average_reward = evaluate_policy(env, optimal_policy)
print("Average Reward:", average_reward)

Optimal Policy: 
[[1 3 1 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 3 3 0]]
Optimal Q-Values:
[[0.02355972 0.02463036 0.02369449 0.02210632]
 [0.01155378 0.01298456 0.0141779  0.02550483]
 [0.02957713 0.03424103 0.02948144 0.02157563]
 [0.0103027  0.01371659 0.00852264 0.02102668]
 [0.03517407 0.02219203 0.01634041 0.0208715 ]
 [0.         0.         0.         0.        ]
 [0.04738669 0.03074347 0.04532827 0.00517788]
 [0.         0.         0.         0.        ]
 [0.02387975 0.04730862 0.0359639  0.05395988]
 [0.04324442 0.09454847 0.06209223 0.0278745 ]
 [0.13926184 0.07814057 0.11960327 0.0310159 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.03254301 0.0625088  0.06063448 0.14145995]
 [0.11595006 0.34819098 0.35298562 0.41853957]
 [0.         0.         0.         0.        ]]
Average Reward: 0.24


Comparing rewards of Q-Learning with and without decaying epsilon

In [ ]:
# Q-Learning without decay
Q_basic, policy_basic = q_learning(env)

# Q-Learning with decay
Q_decay, policy_decay = q_learning2(env)

# Evaluate both policies
average_reward_basic = evaluate_policy(env, policy_basic)
average_reward_decay = evaluate_policy(env, policy_decay)

print("Average Reward with Basic Q-Learning:", average_reward_basic)
print("Average Reward with Q-Learning (Decaying Epsilon):", average_reward_decay)

Average Reward with Basic Q-Learning: 0.19
Average Reward with Q-Learning (Decaying Epsilon): 0.68


I have tested with decay_rate values ranging from 0.95 to 0.5 and the results vary each execution. Sometimes the function without decaying epsilon performs better, but, after numerous executions, it is clear that the decaying epsilon method offers increased performance and provides a greater reward majority of the time.